In [129]:
import pandas as pd 
from pandas import DataFrame
import json 
with open('레시피 기본 정보.json','r',encoding='utf-8') as f:
    json_data = json.load(f)
with open('레시피 재료 정보.json','r',encoding='utf-8') as a:
    json_data2 = json.load(a)
with open('회원2.json','r',encoding='utf-8') as b:
    json_data3 = json.load(b)
    
# ensure_ascii=False를 추가하니 한글로 표현됨 
json.dumps(json_data, ensure_ascii=False,indent=4)
json.dumps(json_data2, ensure_ascii=False,indent=4)
json.dumps(json_data3, ensure_ascii=False, indent=4)

'{\n    "items": [\n        {\n            "mem_id": "회원아이디",\n            "rating": "점수",\n            "img_url": "이미지 URL",\n            "RECIPE_ID": "레시피 ID"\n        }\n    ],\n    "data": [\n        {\n            "mem_id": "a",\n            "rating": 5,\n            "img_url": "abcd",\n            "RECIPE_ID": 1\n        },\n        {\n            "mem_id": "a",\n            "rating": 3,\n            "img_url": "abcd",\n            "RECIPE_ID": 2\n        },\n        {\n            "mem_id": "a",\n            "rating": 3,\n            "img_url": "abcd",\n            "RECIPE_ID": 4\n        },\n        {\n            "mem_id": "a",\n            "rating": 1,\n            "img_url": "abcd",\n            "RECIPE_ID": 3\n        }\n    ]\n}'

In [130]:
# 리스트 출력하는 대로 출력하면 값이 잘 나타남 
recipe = json_data["data"]
recipe2 = json_data2["data"]
member = json_data3["data"]

for i in range(len(recipe)):
    print(recipe[i]['RECIPE_NM_KO'])
    print(recipe[i]['TY_CODE'])
    print(recipe[i]['SUMRY'])
    print(recipe[i]['DET_URL']) # 여기에는 크롤링한 URL 추가 
    print()
    
    #spring으로 구현할 수 있을지?

나물비빔밥
3010001
육수로 지은 밥에 야채를 듬뿍 넣은 영양만점 나물비빔밥!
http://www.okdab.com/consumer/recipe/recipeView.do?recipeSn=1

오곡밥
3010001
정월대보름에 먹던 오곡밥! 영양을 한그릇에 담았습니다.
http://www.okdab.com/consumer/recipe/recipeView.do?recipeSn=2

잡채밥
3010001
잡채밥 한 그릇이면 오늘 저녁 끝! 입 맛 없을 때 먹으면 그만이지요~
http://www.okdab.com/consumer/recipe/recipeView.do?recipeSn=3

콩나물밥
3010001
다이어트에 으뜸인 콩나물밥. 밥 물 넣을때 평소보다 적게 넣는거 잊지마세요!
http://www.okdab.com/consumer/recipe/recipeView.do?recipeSn=4

약식
3010013
집에서도 쉽게 만들어 맛있게 먹을 수 있답니다. 어려워 마시고 만들어 보세요~!
http://www.okdab.com/consumer/recipe/recipeView.do?recipeSn=5

호박죽
3010001
호박죽 한 그릇이면 하루가 든든하답니다.
http://www.okdab.com/consumer/recipe/recipeView.do?recipeSn=6

흑임자죽
3010001
검은깨를 갈아서 만든 고소함이 가득한 흑임자죽. 남녀노소 모두 사랑하는 맛!
http://www.okdab.com/consumer/recipe/recipeView.do?recipeSn=7

카레라이스
3010001
향긋한 카레향이 너무 좋지요. 누구나 좋아하는 만들기도 간편한 음식입니다.
http://www.okdab.com/consumer/recipe/recipeView.do?recipeSn=8

오므라이스
3010001
각종 채소를 계란 속에 꼭꼭 숨겨 편식하는 아이들도 맛있게 먹어요~
http://www.okdab.com/consumer/recipe/recipeV

In [131]:
result_df = DataFrame(recipe,columns=['RECIPE_ID','RECIPE_NM_KO','TY_CODE','TY_NM','RATING'])
result_df2 = DataFrame(recipe2, columns=['RECIPE_ID','IRDNT_NM'])
result_df3 = DataFrame(member, columns=['mem_id','rating','RECIPE_ID'])

print(result_df.head())
print(result_df2.head())
print(result_df3.head())

   RECIPE_ID RECIPE_NM_KO  TY_CODE TY_NM  RATING
0          1        나물비빔밥  3010001     밥     5.0
1          2          오곡밥  3010001     밥     4.0
2          3          잡채밥  3010001     밥     3.0
3          4         콩나물밥  3010001     밥     2.0
4          5           약식  3010013  떡/한과     1.0
   RECIPE_ID IRDNT_NM
0          1        쌀
1          1       안심
2          1      콩나물
3          1      청포묵
4          1      미나리
  mem_id  rating  RECIPE_ID
0      a       5          1
1      a       3          2
2      a       3          4
3      a       1          3


In [135]:
data = pd.merge(result_df,result_df2, on='RECIPE_ID', how='outer')
data.head()

data = pd.merge(data,result_df3, on ='RECIPE_ID', how='outer')
data.head()

matrix=data.pivot_table(index='mem_id',columns='RECIPE_NM_KO',
                       values='rating')
matrix.head()

RECIPE_NM_KO,나물비빔밥,오곡밥,잡채밥,콩나물밥
mem_id,,,,
a,5.0,3.0,1.0,3.0


In [14]:
MW = 0.1 

def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np,sum(s1_c * s2_c)/np.sqrt(np.sum(s1_c ** 2) *np.sum(s2_c **2))

def recommend(input_food, matrix, n, similar_method=True):
    input_method = json_data[json_data['RECIPE_NM_KO']== input_food]['method'].iloc(0)[0]
    
    result = []
    for name in matrix.columns:
        if name == RECIPE_NM_KO:
            continue
    cor = pearsonR(matrix[RECIPE_NM_KO], matrix[name])

    if similar_method and len(input_method)> 0:
        temp_method = json_data[json_data['RECIPE_NM_KO'] == name]['method'].iloc(0)[0]
        
        same_count = np.sum(np.isin(input_method, temp_method))
        cor += (MW * same_count)
        
        if np.isnan(cor):
            continue
        else:
            result.append((name, '{:2f}'.format(cor),temp_method))
        
    result.sort(key=lambda r: r[1], reverse=True)
    
    return result[:n]



SyntaxError: 'continue' not properly in loop (<ipython-input-14-01917de54379>, line 27)